In [1]:
import sqlite3

# Membuat koneksi ke database SQLite
conn = sqlite3.connect('news_data.db')
c = conn.cursor()

# Membuat tabel untuk menyimpan teks yang diekstrak
c.execute('''
CREATE TABLE IF NOT EXISTS extracted_text (
    url TEXT,
    content TEXT,
    FOREIGN KEY (url) REFERENCES href (id)
)
''')
conn.commit()

In [2]:
import requests
from lxml import html

In [3]:
c.execute('SELECT url FROM news WHERE url not in (select url from extracted_text)')
urls = c.fetchall()

In [4]:
from tqdm import tqdm

import warnings

warnings.simplefilter('ignore')

In [5]:
start = 0

In [6]:
for i in tqdm(range(len(urls))):
    if i < start:
        continue
      
    try:
        url = urls[i]
    
        current_url = url[0]
        all_paragraphs = []  # List untuk menyimpan semua paragraf
        response = requests.get(f'{current_url}', verify=False)
        tree = html.fromstring(response.content)
        total_page_link = tree.xpath('//a[contains(text(),"Next")]/../preceding-sibling::li[1]//a/text()')
        if total_page_link != []:
            total_page_link = int("".join(total_page_link))
        else:
            total_page_link = 1
    
        for page in range(1, total_page_link+1):
            response = requests.get(f'{current_url}?page={page}', verify=False)
            if response.status_code == 200:
                tree = html.fromstring(response.content)
                paragraphs = tree.xpath("//div[@class='page-content']//div[@itemprop='articleBody']//p//text()")
                all_paragraphs.extend(paragraphs)  # Menambahkan paragraf dari halaman saat ini
            else:
                print("Failed to retrieve the page:", current_url)
                break
    
        # Menggabungkan semua paragraf menjadi satu teks panjang
        full_text = ' '.join(all_paragraphs)
        
        c.execute('INSERT INTO extracted_text (url, content) VALUES (?, ?)', (url[0], full_text))
        conn.commit()

    except Exception as e:
        print(current_url)
        print(str(e))
        continue
            

100%|████████████████████████████████████████████████████████████████████████████████| 420/420 [15:01<00:00,  2.15s/it]
